In [12]:
import pandas as pd
data =  pd.read_csv("d:/data/HR.csv") 
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary_type
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Departments            14999 non-null  object 
 9   salary_type            14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [25]:
df1=data
#df1["number_project"]=df1["number_project"].astype('object')
#df1["Work_accident"]=df1["Work_accident"].astype('object')
df1["left"]=df1["left"].astype('object')
#df1["promotion_last_5years"]=df1["promotion_last_5years"].astype('object')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  object 
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  object 
 6   left                   14999 non-null  object 
 7   promotion_last_5years  14999 non-null  object 
 8   Departments            14999 non-null  object 
 9   salary_type            14999 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 1.1+ MB


In [26]:
from pycaret.classification import *

In [27]:
s = setup(df1,target="left",session_id=124)

,Description,Value
0,session_id,124
1,Target,left
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(14999, 10)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [28]:
df1.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary_type
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Fit the best model

In [29]:
best = compare_models(n_select=15)
compare_model_results = pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9887,0.9914,0.9615,0.9906,0.9757,0.9684,0.9686,0.1160
lightgbm,Light Gradient Boosting Machine,0.9849,0.9931,0.9538,0.9820,0.9676,0.9577,0.9580,0.0970
et,Extra Trees Classifier,0.9824,0.9903,0.9558,0.9696,0.9626,0.9511,0.9512,0.1210
dt,Decision Tree Classifier,0.9783,0.9728,0.9623,0.9473,0.9546,0.9404,0.9405,0.0200
gbc,Gradient Boosting Classifier,0.9764,0.9868,0.9277,0.9715,0.9490,0.9336,0.9342,0.1490
ada,Ada Boost Classifier,0.9591,0.9813,0.9016,0.9245,0.9129,0.8862,0.8864,0.0620
knn,K Neighbors Classifier,0.9334,0.9691,0.9049,0.8301,0.8658,0.8217,0.8231,0.2360
lr,Logistic Regression,0.8958,0.9362,0.7908,0.7747,0.7826,0.7141,0.7142,0.4320
ridge,Ridge Classifier,0.8807,0.0000,0.7973,0.7265,0.7601,0.6809,0.6823,0.0170
lda,Linear Discriminant Analysis,0.8800,0.9277,0.8270,0.7133,0.7659,0.6858,0.6892,0.0240


In [30]:
best[0]

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=124, verbose=0,
                       warm_start=False)

#### creating a predict function to be passed into gradio UI

In [31]:
df1.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'Departments', 'salary_type'],
      dtype='object')

In [36]:
def predict(model, satisfaction_level, last_evaluation, number_project, average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Departments,salary_type):
  
    df = pd.DataFrame.from_dict({'satisfaction_level': [satisfaction_level], 'last_evaluation': [last_evaluation],
                                 'number_project': [number_project], 'average_montly_hours': [average_montly_hours],
                                 'time_spend_company': [time_spend_company], 'Work_accident': [Work_accident],
                                 'promotion_last_5years': [promotion_last_5years], 'Departments': [Departments], 
                                 'salary_type': [salary_type]})
    
    model_index = list(compare_model_results['Model']).index(model)
    model = best[model_index]
    pred = predict_model(model, df, raw_score=True)
    return {'left_No': pred[0].astype('float64'), 
            'left_Yes': pred[1].astype('float64')}

## Creating an Interface with gradio

In [37]:
import gradio as gr

model = gr.inputs.Dropdown(list(compare_model_results['Model']), label="Model")

satisfaction_level = gr.inputs.Slider(minimum=0, maximum=1, default=df1['satisfaction_level'].mean(), label = 'satisfaction_level')

last_evaluation = gr.inputs.Slider(minimum=0, maximum=1, default=df1['last_evaluation'].mean(), label = 'last_evaluation')

number_project = gr.inputs.Dropdown(list(df1['number_project'].unique()), label ='number_project')

average_montly_hours = gr.inputs.Slider(minimum=1, maximum=400, default=df1['average_montly_hours'].mean(), label = 'average_montly_hours')

time_spend_company = gr.inputs.Slider(minimum=1, maximum=10, default=df1['time_spend_company'].mean(), label = 'time_spend_company')

Work_accident =  gr.inputs.Dropdown(list(df1['Work_accident'].unique()), label ="Work_accident")

promotion_last_5years = gr.inputs.Dropdown(list(df1['promotion_last_5years'].unique()), label ='promotion_last_5years')

Departments = gr.inputs.Dropdown(list(df1['Departments'].unique()), label='Departments')

salary_type = gr.inputs.Dropdown(list(df1['salary_type'].unique()), label ='salary_type')

gr.Interface(predict, [model,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Departments,salary_type], "label", live=True).launch()

Running locally at: http://127.0.0.1:7899/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7899/', None)

[2021-09-13 19:36:13,901] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2895, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch